In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# 读入基础数据
## 确定是读入训练集进行训练，还是读入测试集生成提交结果
### 仅训练集
rawData = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\train.csv')
### 训练集和测试集。注意这里把训练集也读入了，为了dummy的时候划分一致。后续需要删除训练集数据
# rawData = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\train.csv')
# rawData.drop(['conversionTime'],axis=1,inplace=True) #先去除conversionTime
# rawData.insert(0, 'instanceID', pd.Series(np.zeros(rawData['label'].count())))#插入instanceID使append不会乱序
# rawData = pd.concat([rawData,pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\test.csv')])
## 将部分数据读入
rawData['date'] = rawData['clickTime'].apply(lambda x: int(x/10000))
rawData = pd.merge(rawData, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\user.csv'), on='userID')
rawData = pd.merge(rawData, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\ad.csv'), on='creativeID')
rawData = pd.merge(rawData, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\position.csv'), on='positionID')

In [ ]:
rawData[rawData['label']==0].count()

In [ ]:
# 对点击时间进行处理
def dealTime(clicktime):
    hour = int(str(clicktime)[2:4])
    minute = int(str(clicktime)[4:6])
    x = int(60*hour+minute)/30
    if x in [18,19,20]:
        return 1
    elif x in [21,22,23]:
        return 2
    elif x in [4,24]:
        return 3
    elif x in [25,26]:
        return 4
    elif x in [2,3,6,10,27,31,47]:
        return 5
    elif x in [0,1,5,7,8,9,11,28,30,44,45]:
        return 6
    elif x in [13,14]:
        return 8
    elif x in [15]:
        return 9
    else: #中下的最多，所以else来处理
        return 7

In [ ]:
# 读入更多特征
## 读入user_single_vote
# rawData = pd.merge(rawData, 
#                    pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\user_single_vote.csv')[['userID','appID','voteMinus']].drop_duplicates(), 
#                    on=['userID', 'appID'])
## 读入age
rawData.drop(['age'], axis=1, inplace=True) #有些原有字段中有的但经过处理的，先删除，再读入
rawData = pd.merge(rawData, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\age.csv'), on='userID')
## 读入appCategory
rawData = pd.merge(rawData, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\appCategory.csv'), on='creativeID')
## 读入clickTime_hour
rawData['clickTime_hour'] = rawData['clickTime'].apply(dealTime)
## 读入lastNCVR_advertiserID
rawData = pd.merge(rawData, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\lastNCVR_advertiserID.csv'), on=['date', 'creativeID'])
## 读入lastNCVR_appID
rawData = pd.merge(rawData, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\lastNCVR_appID.csv'), on=['date', 'creativeID'])
## 读入lastNCVR_positionType
rawData = pd.merge(rawData, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\lastNCVR_positionType.csv'), on=['date', 'positionID'])
## 读入lastNCVR_sitesetID
rawData = pd.merge(rawData, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\lastNCVR_sitesetID.csv'), on=['date', 'positionID'])

In [ ]:
# # 将user_single_vote离散化
# # 获得targetID名单
# dataAppID = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\ad.csv')
# appIDs = set(dataAppID['appID'])
# # 为每个appID单独生成字段
# for id in appIDs:
#     rawData['vm' + str(id) + '_1'] = (rawData['appID']==id).astype(int) * rawData['voteMinus']
#     rawData['vm' + str(id) + '_0'] = (rawData['voteMinus']==0).astype(int)

In [ ]:
rawData.columns

In [ ]:
# # # 特征哑变量化与离散化
import sklearn.preprocessing as preprocessing
## 离散变量哑变量化
def featureDummy(features):
    for feature in features:
        global rawData #声明data是global而不是局部变量
        dummies = pd.get_dummies(rawData[feature], prefix=feature)
        rawData = pd.concat([rawData, dummies], axis=1)
        rawData.drop([feature], axis=1, inplace=True)
## 连续变量标准化
dataScaleTrain = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\train.csv') #用train的数据来求scale
dataScaleTrain['date'] = dataScaleTrain['clickTime'].apply(lambda x: int(x/10000))
dataScaleTrain = pd.merge(dataScaleTrain, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\lastNCVR_advertiserID.csv'), on=['date', 'creativeID'])
dataScaleTrain = pd.merge(dataScaleTrain, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\lastNCVR_appID.csv'), on=['date', 'creativeID'])
dataScaleTrain = pd.merge(dataScaleTrain, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\lastNCVR_positionType.csv'), on=['date', 'positionID'])
dataScaleTrain = pd.merge(dataScaleTrain, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\lastNCVR_sitesetID.csv'), on=['date', 'positionID'])
def featureScale(features):
    for feature in features:
        global rawData #声明data是global而不是局部变量
        scaler = preprocessing.StandardScaler()
        scaleParam = scaler.fit(dataScaleTrain[feature])
        rawData[feature] = scaler.fit_transform(rawData[feature], scaleParam)

# 哑变量化与离散化
featureDummy(['connectionType', 'telecomsOperator', 'gender','marriageStatus','haveBaby', 'appPlatform', 
              'education', 'sitesetID','positionType','appCategory', 'age', 'clickTime_hour'])
featureScale(['lastNCVR_appID', 'lastNCVR_advertiserID','lastNCVR_positionType', 'lastNCVR_sitesetID'])
# 删除不再使用的数据，节省内存
del dataScaleTrain

In [ ]:
# 提取特征
## 筛掉一些字段
data = rawData.drop(['clickTime', 'conversionTime', 'creativeID', 'userID', 'positionID', 
                     'date', 'hometown', 'residence','advertiserID', 'adID', 'camgaignID','appID'], axis=1)
# data = rawData.drop(['clickTime', 'creativeID', 'userID', 'positionID', 
#                      'date', 'hometown', 'residence','advertiserID', 'adID', 'camgaignID','appID'], axis=1)
# data = rawData.filter(regex='label|vm+')
# data = rawData.filter(regex='instanceID|label|voteMinus')

In [ ]:
print len(list(data.columns))

# 训练分类器

In [ ]:
# 平衡正反例
def balance(train_pro):
    ## 从正例中随机取出比例为a的数据，划分
    train_size_p = int(data[data['label']==1]['label'].count()*train_pro)
    train_size_n = int(train_size_p * 39.20424181) #原始样本中负例是正例的这个倍数
    sampler1 = np.random.permutation(data[data['label']==1]['label'].count())
    index1 = data[data['label']==1].index[sampler1] #重排序后的索引
    train1 = data[data['label']==1].ix[index1[:train_size_p]]
    test1 = data[data['label']==1].ix[index1[train_size_p:]]
    ## 从负例中随机取出相同数量的数据，划分
    sampler0 = np.random.permutation(data[data['label']==0]['label'].count())
    index0 = data[data['label']==0].index[sampler0] #重排序后的索引
    train0 = data[data['label']==0].ix[index0[:train_size_n]]
    test0 = data[data['label']==0].ix[index0[train_size_n:]]
    ## 构建训练集和测试集
    data_train = pd.concat([train1, train0])
    data_test = pd.concat([test1, test0])
    return data_train.as_matrix()[:,1:], data_test.as_matrix()[:,1:], data_train.as_matrix()[:,0],  data_test.as_matrix()[:,0]
# 获取正反例
trainX, testX, trainY, testY = balance(0.2)

In [ ]:
# 划分训练集和测试集
from sklearn import cross_validation
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
# X = data.as_matrix()[:,1:]
# y = data.as_matrix()[:,0]
# trainX, testX, trainY, testY = cross_validation.train_test_split(X, y, test_size=0.2, random_state=174)
# y
# 训练
clf = linear_model.LogisticRegressionCV(Cs=10, penalty='l2', tol=1e-4, n_jobs=-1, cv=5)
# clf = RandomForestClassifier()
clf.fit(trainX, trainY)
print clf

In [ ]:
# 评估模型结果
import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll
from sklearn import metrics
predict = clf.predict(testX)
print 'P&R'
print metrics.classification_report(testY, predict)
print 'loss:'
predict_pro = clf.predict_proba(testX)[:, 1]
print logloss(testY, predict_pro)

In [ ]:
cf = pd.DataFrame({'columns':data.columns[1:], 'coef':list(clf.coef_.T)})
cf.sort_values(by=['coef'],ascending=False, inplace=True)

In [ ]:
# print cf.head()
# print cf.tail()
cf

In [ ]:
cf.to_csv('C:\\Users\\work\\Desktop\\tencent\\clf\\coef_4.csv',index=False)

In [ ]:
predict_pro

In [ ]:
# 保存分类器
from sklearn.externals import joblib
joblib.dump(clf, 'C:\\Users\\work\\Desktop\\tencent\\clf\\4.pkl', compress=3)

# 预测

In [ ]:
# 读取分类器
from sklearn.externals import joblib
clf = joblib.load('C:\\Users\\work\\Desktop\\tencent\\clf\\3.pkl')

In [ ]:
# 提取特征并预测
## 只选择instanceID>0的记录，并重排序instanceID
dataPredict = data[data.instanceID>0].sort_values(by='instanceID')

In [ ]:
dataPredict.head()

In [ ]:
X = dataPredict.as_matrix()[:,2:]
y = clf.predict_proba(X)

In [ ]:
output = pd.DataFrame({'instanceID':range(1,dataPredict['instanceID'].count()+1),'prob':y[:, 1]})

In [ ]:
output.to_csv('C:\\Users\\work\\Desktop\\tencent\\predict\\2.csv', index=False)

In [ ]:
d = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\predict\\2.csv')

In [ ]:
d.count()